In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Remove index_col if there's no extra column or index column at the starting
df_common=pd.read_csv("/content/drive/MyDrive/gene_ext/sub/06/common_wd6.csv", index_col="Unnamed: 0")
df_protein_file = pd.read_csv("/content/drive/MyDrive/gene_ext/ind_protein/ind_chr5_prot.csv")

In [ ]:
df_common.shape

(134491, 12)

In [ ]:
# df_common.columns

In [ ]:
df_common

,#CHROM,POS,REF,ALT,chr.r,startpos.r,endpos.r,seq.r,seq.q,startpos.q,endpos.q,chr.q
0,NC_037328.1,666159,A,c,NC_037328.1,666159,666159,A,c,4258,4258,NC_032650.1
1,NC_037328.1,667388,T,C,NC_037328.1,667388,667388,T,C,5488,5488,NC_032650.1
2,NC_037328.1,667947,G,a,NC_037328.1,667947,667947,G,a,6045,6045,NC_032650.1
3,NC_037328.1,668612,C,T,NC_037328.1,668612,668612,C,T,6710,6710,NC_032650.1
4,NC_037328.1,669280,G,A,NC_037328.1,669280,669280,G,A,7379,7379,NC_032650.1
...,...,...,...,...,...,...,...,...,...,...,...,...
177854,NC_037328.1,157552506,C,T,NC_037328.1,157552506,157552506,C,T,161074678,161074678,NC_032650.1
177855,NC_037328.1,157555075,C,A,NC_037328.1,157555075,157555075,C,A,161077248,161077248,NC_032650.1
177856,NC_037328.1,157556980,C,A,NC_037328.1,157556980,157556980,C,A,161079153,161079153,NC_032650.1
177857,NC_037328.1,157556981,G,A,NC_037328.1,157556981,157556981,G,A,161079154,161079154,NC_032650.1


In [ ]:
df_protein_file.shape

(2001, 11)

In [ ]:
def map_keywords(str_sentence):
    list_keywords = [
        "immunoglobulin","immunoreceptor","autoimmune","TLR","IgG",
        "autoimmune","autophagy","immunogen","immune","innate","T-cell","NF-kappa", "antigen",
        "B-cell","lymphocyte","histocompatibility","CD24","CD4","LY96", "BCR",
        "IFIT3","PGLYRP1","NKG2D","UL16","leukocyte","cytokine", "interleukin","interferon"
        "antimicrobial peptide","beta-defensin 2","IL16","IL2","chemokine", "antibody"
    ]
    bool_found = bool(
        [
            i for i in list_keywords if i.lower() in str_sentence.lower()
        ]
    )
    return bool_found

In [ ]:
df_protein_file["keyword_found"] = df_protein_file.apply(lambda row: map_keywords(row["Protein Name"]),axis=1)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].shape

(54, 12)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates().shape

(54, 12)

In [ ]:
# df_protein_file_imm=df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates()

In [ ]:
df_protein_file["start_of_next_gene"] = df_protein_file["Start"].shift(-1).fillna(0)

In [ ]:
# df_protein_file.head()

In [ ]:
df_protein_file.to_csv("protein1.csv")

In [ ]:
a=0
b=0
c=0
d=0
e=0
f=0

In [ ]:
from tqdm import tqdm

In [ ]:
final_df = pd.DataFrame(columns=list(df_protein_file.columns)+["gene_type"]+["startpos.q"]+["endpos.q"]+["seq.r"]+["seq.q"]+["LOV"])
for i, row in tqdm(df_common.iterrows()):
    # Check for the at start
    if df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_start"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        a=a+1
    # check for the at end
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_end"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        b=b+1
    # Check for the within gene
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "within_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        c=c+1
    # Check for the gene_start_with variation end
    elif df_protein_file[(df_protein_file["Start"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "gene_start_with variation end"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        d=d+1
    # Check for the gene_stop_ with variation start
    elif df_protein_file[(df_protein_file["Stop"]==row["startpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]==row["startpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "gene_stop_ with variation start"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        e=e+1
    # Check for the intergenic gene
    elif df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "intergenic_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        f=f+1
    else:
        pass


102471it [2:30:03,  5.88it/s]

In [ ]:
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

In [ ]:
final_df.shape

In [ ]:
final_df.drop_duplicates().shape

In [ ]:
final_df.to_csv("genesv1.csv")

In [ ]:
# non coding region
final_df1 = pd.DataFrame(columns=df_common.columns)
for index, row in tqdm(df_common.iterrows()):
    # Check if 'startpos.q' is present in any row of file2
    if row['startpos.q'] not in final_df['startpos.q'].values:
        # Append the row to the DataFrame if not found
        final_df1 = final_df1.append(row, ignore_index=True)

# Save the DataFrame with rows not found in file2 to a new CSV file
final_df1.to_csv("non_coding1.csv", index=False)

In [ ]:
final_df1.shape

In [ ]:
final_df1.drop_duplicates().shape

In [ ]:
imm = pd.DataFrame
imm=final_df[final_df["keyword_found"]==True]

In [ ]:
imm.shape

In [ ]:
imm.drop_duplicates().shape

In [ ]:
imm.drop_duplicates().to_csv("immv1.csv")

In [ ]:
# counting the values
# Create a new DataFrame with unique combinations of 'Locus', 'startpos.q', and 'endpos.q'
locus_counts = final_df.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts.shape

In [ ]:
locus_counts.to_csv("locus_counts1.csv")

In [ ]:
locus_counts_imm = imm.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts_imm.shape

In [ ]:
locus_counts_imm.to_csv("locus_counts_imm1.csv")

In [ ]:
gene_count=locus_counts.groupby(["gene_type"]).size().reset_index(name="count")

In [ ]:
gene_count

In [ ]:
gene_count.to_csv("gene_count1.csv")

In [ ]:
loc_imm_count=locus_counts_imm.groupby(by=["Locus","gene_type"]).size().reset_index(name="count")
# loc_imm_count.columns=["Locus", "gene_type", "count"]

loc_imm_count.to_csv("immlocus_gene_type_counts1.csv", index=False)

In [ ]:
loc_imm_count

In [ ]:
# Calculate value counts
count = locus_counts["gene_type"].value_counts().reset_index()
count.columns = ["gene_type", "Count"]
count.loc[len(count.index)]=["immune_gene", len(locus_counts_imm)]
count.loc[len(count.index)]=["non_coding_gene", len(final_df1)]


# Save the result to a CSV file
count.to_csv("subcounts1.csv", index=False)

In [ ]:
import shutil

shutil.copy("protein1.csv","/content/drive/MyDrive/gene_ext/sub/01/new")
shutil.copy("genesv1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("immv1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("locus_counts1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("locus_counts_imm1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("immlocus_gene_type_counts1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("gene_count1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")
shutil.copy("subcounts1.csv","/content/drive/MyDrive/gene_ext/sub/01/new/")

In [ ]:
# extract within gene information
import pandas as pd
loc_count=pd.read_csv("/content/drive/MyDrive/gene_ext/del/1/new/locus_counts1.csv")
# loc_count=pd.read_csv("/content/drive/MyDrive/gene_ext/ins/1/new/locus_counts1.csv")

In [ ]:
loc_count.shape

In [ ]:
locus_count_within_gene=loc_count[loc_count["gene_type"]=="within_gene"]
locus_count_inter_gene=loc_count[loc_count["gene_type"]=="intergenic_gene"]


In [ ]:
locus_count_within_gene.shape

In [ ]:
locus_count_inter_gene.shape

In [ ]:
# Create a new DataFrame with unique combinations of 'Locus', 'startpos.q', and 'endpos.q'
unique_counts = locus_count_within_gene.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])
counts=unique_counts.groupby(["Locus"]).size().reset_index(name="count")

counts.to_csv("loc_subs_counts1.csv")
loc50_subs=counts[counts["count"]>=50]

In [ ]:
loc50_subs=counts[counts["count"]>=50]

In [ ]:
loc50_subs.shape

In [ ]:
locus_count_within_gene.to_csv("locus_within_gene1.csv")
locus_count_inter_gene.to_csv("locus_inter_gene1.csv")
loc50_subs.to_csv("loc50_subs1.csv")

In [ ]:
# immune genes information - deletion
import shutil

Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]
# Chr_list = [1]
for chr in Chr_list:
  print(f"chromosome:{chr}")
  in_file1=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/sub/{chr}/new/immloc_count_within_gene{chr}.csv"
  outloc=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/sub/{chr}/new"
  imf1=pd.read_csv(in_file1)

  # extract the immune has how many variations and its gene type
  locwithin_imm_count=imf1.groupby(by=["Locus"]).size().reset_index(name="count")

  # Group by 'Locus' and calculate the sum of 'LOV'
  grouped = imf1.groupby('Locus')['LOV'].sum().reset_index()

  # Merge the two files on the 'Locus' column
  merged_df = pd.merge(locwithin_imm_count, grouped, on='Locus')

  # Rename columns for clarity
  merged_df.rename(columns={'count': 'No. of Variations', 'LOV': 'Length of Variations'}, inplace=True)

  chr_col=f"{chr}"

  merged_df["Chromosome"] = chr_col

  # Reorder columns
  reorder_df = merged_df[['Chromosome','Locus', 'No. of Variations', 'Length of Variations']]

  out_f1=f"imm_var_count{chr}.csv"
  reorder_df.to_csv(out_f1)

  shutil.copy(out_f1, outloc)
